In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.preprocessing import Normalizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
spaceship = pd.read_csv('train.csv')
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [4]:
# maybe don't drop Cabin, it could be useful
spaceship.drop(['Name', 'Cabin'],axis=1,inplace=True)

In [5]:
spaceship.fillna({
    "CryoSleep": 0, "Age": 0, "VIP": 0, "RoomService": 0, 
    "FoodCourt": 0, "ShoppingMall": 0, "Spa": 0, "VRDeck": 0, 
    "Destination": "Unknown Destination", "HomePlanet": "Unknown HomePlanet",
}, inplace=True)

In [6]:
# check for missing values
spaceship.isnull().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64

In [7]:
spaceship.Destination.value_counts()

TRAPPIST-1e            5915
55 Cancri e            1800
PSO J318.5-22           796
Unknown Destination     182
Name: Destination, dtype: int64

In [8]:
spaceship.HomePlanet.value_counts()

Earth                 4602
Europa                2131
Mars                  1759
Unknown HomePlanet     201
Name: HomePlanet, dtype: int64

In [9]:
spaceship_categorial = pd.get_dummies(spaceship[['Destination', 'HomePlanet']])
spaceship_categorial.head()

,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_Unknown Destination,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_Unknown HomePlanet
0,0,0,1,0,0,1,0,0
1,0,0,1,0,1,0,0,0
2,0,0,1,0,0,1,0,0
3,0,0,1,0,0,1,0,0
4,0,0,1,0,1,0,0,0


In [10]:
spaceship.drop(['Destination', 'HomePlanet'],axis=1,inplace=True)
spaceship = pd.concat([spaceship, spaceship_categorial], axis=1)

In [11]:
spaceship.head()

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_Unknown Destination,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_Unknown HomePlanet
0,0001_01,False,39.0,False,0.0,0.0,0.0,0.0,0.0,False,0,0,1,0,0,1,0,0
1,0002_01,False,24.0,False,109.0,9.0,25.0,549.0,44.0,True,0,0,1,0,1,0,0,0
2,0003_01,False,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,0,0,1,0,0,1,0,0
3,0003_02,False,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,0,0,1,0,0,1,0,0
4,0004_01,False,16.0,False,303.0,70.0,151.0,565.0,2.0,True,0,0,1,0,1,0,0,0


In [12]:
# separate data to features and labels
X_train = spaceship.drop('Transported',axis=1)
y_train = spaceship['Transported']

In [13]:
# normalize the data
transformer = Normalizer().fit(X_train)
X_train = transformer.transform(X_train)

In [14]:
# train the model using gridsearchcv
rfc = RandomForestClassifier()

search_best = GridSearchCV(rfc, {
    'n_estimators': range(5, 30, 5),
    'max_depth': range(5, 30, 5),
    'min_samples_split': range(50, 200, 50),
    'min_samples_leaf': [1, 2, 5, 10],
}, cv=5, return_train_score=False)
search_best.fit(X_train, y_train)
search_best.best_params_

KeyboardInterrupt: 

In [15]:
random_forest = RandomForestClassifier(
    n_estimators=20, max_depth=20, min_samples_split=100, min_samples_leaf=2
)
random_forest.fit(X_train, y_train)
random_forest.score(X_train, y_train)

0.8164040032209824

In [16]:
spaceship_test = pd.read_csv('test.csv')
spaceship_test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [17]:
spaceship_test.drop(['Name', 'Cabin'],axis=1,inplace=True)

In [18]:
spaceship_test_categorical = pd.get_dummies(spaceship_test[['Destination', 'HomePlanet']])
spaceship_test = pd.concat([spaceship_test, spaceship_test_categorical], axis=1)
spaceship_test.drop(['Destination', 'HomePlanet'],axis=1,inplace=True)
spaceship_test.head()

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars
0,0013_01,True,27.0,False,0.0,0.0,0.0,0.0,0.0,0,0,1,1,0,0
1,0018_01,False,19.0,False,0.0,9.0,0.0,2823.0,0.0,0,0,1,1,0,0
2,0019_01,True,31.0,False,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0
3,0021_01,False,38.0,False,0.0,6652.0,0.0,181.0,585.0,0,0,1,0,1,0
4,0023_01,False,20.0,False,10.0,0.0,635.0,0.0,0.0,0,0,1,1,0,0


In [20]:
spaceship_test.isnull().sum()

PassengerId                    0
CryoSleep                     93
Age                           91
VIP                           93
RoomService                   82
FoodCourt                    106
ShoppingMall                  98
Spa                          101
VRDeck                        80
Destination_55 Cancri e        0
Destination_PSO J318.5-22      0
Destination_TRAPPIST-1e        0
HomePlanet_Earth               0
HomePlanet_Europa              0
HomePlanet_Mars                0
dtype: int64

In [19]:
y_pred = random_forest.predict(spaceship_test)
y_pred

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').